# Parameter-Efficient Fine-tuning of Whisper for Arabic Dialects using PEFT & LoRA

This notebook demonstrates how to fine-tune Whisper models for Arabic dialects using Parameter-Efficient Fine-Tuning (PEFT) with Low-Rank Adaptation (LoRA). This approach:

1. **Reduces memory usage**: Fine-tune large models with less GPU memory
2. **Faster training**: Only trains 1% of the model parameters
3. **Better generalization**: Prevents catastrophic forgetting
4. **Smaller checkpoints**: Model adapters are ~60MB vs full model ~1.5GB

We'll fine-tune Whisper-small on Arabic dialects using the MASC dataset with LoRA adapters.

## Install Required Packages

Install the necessary packages for PEFT fine-tuning including bitsandbytes for 8-bit training and PEFT for LoRA adapters.

In [ ]:
# Install required packages for PEFT fine-tuning
!pip install --upgrade pip
!pip install -q transformers datasets librosa evaluate jiwer gradio bitsandbytes==0.41.3 accelerate
!pip install -q peft>=0.7.0

## GPU Setup and Environment Check

In [ ]:
# Check GPU availability and specs
import torch
import os

print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")
    print(f"CUDA memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")

# Set environment for CUDA
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

## Hugging Face Authentication

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

## Configuration

In [ ]:
# Model and training configuration
model_name_or_path = "openai/whisper-small"
language = "Arabic"
task = "transcribe"

# Arabic dialect to fine-tune on
# Options: "egyptian", "gulf", "iraqi", "levantine", "maghrebi", "all"
target_dialect = "egyptian"  # Change this to your desired dialect

print(f"Fine-tuning Whisper-small for {target_dialect} Arabic dialect using PEFT/LoRA")

## Load Arabic Dialect Dataset

Load the preprocessed Arabic dialect dataset. In practice, you would replace this with your actual dataset loading logic.

In [ ]:
from datasets import load_dataset, DatasetDict, load_from_disk
import os

# For demonstration, we'll use Common Voice Arabic
# In your actual implementation, replace this with your dialect dataset
try:
    # Try to load preprocessed dialect data if available
    if target_dialect == "all":
        print("Loading all Arabic dialects...")
        # This would be your actual dialect data loading logic
        arabic_dialects = load_dataset("mozilla-foundation/common_voice_11_0", "ar", split="train[:1000]")
        test_data = load_dataset("mozilla-foundation/common_voice_11_0", "ar", split="test[:200]")
    else:
        print(f"Loading {target_dialect} dialect dataset...")
        # This would be your actual dialect data loading logic
        arabic_dialects = load_dataset("mozilla-foundation/common_voice_11_0", "ar", split="train[:1000]")
        test_data = load_dataset("mozilla-foundation/common_voice_11_0", "ar", split="test[:200]")
    
    # Create dataset dict
    dialect_dataset = DatasetDict({
        "train": arabic_dialects,
        "test": test_data
    })
    
    print(f"Dataset loaded: {dialect_dataset}")
    
except Exception as e:
    print(f"Could not load dialect dataset: {e}")
    print("Using Common Voice Arabic as fallback...")
    
    dialect_dataset = DatasetDict()
    dialect_dataset["train"] = load_dataset("mozilla-foundation/common_voice_11_0", "ar", split="train[:1000]", use_auth_token=True)
    dialect_dataset["test"] = load_dataset("mozilla-foundation/common_voice_11_0", "ar", split="test[:200]", use_auth_token=True)
    
    print(f"Fallback dataset loaded: {dialect_dataset}")

## Prepare Feature Extractor, Tokenizer and Processor

In [ ]:
from transformers import WhisperFeatureExtractor, WhisperTokenizer, WhisperProcessor

# Initialize feature extractor
feature_extractor = WhisperFeatureExtractor.from_pretrained(model_name_or_path)

# Initialize tokenizer for Arabic
tokenizer = WhisperTokenizer.from_pretrained(model_name_or_path, language=language, task=task)

# Initialize processor
processor = WhisperProcessor.from_pretrained(model_name_or_path, language=language, task=task)

print("Feature extractor, tokenizer, and processor initialized")

## Data Preprocessing

In [ ]:
from datasets import Audio

# Remove unnecessary columns (keep only audio and sentence)
if "common_voice" in str(type(dialect_dataset["train"])):
    # Remove Common Voice specific columns
    columns_to_remove = ["accent", "age", "client_id", "down_votes", "gender", "locale", "path", "segment", "up_votes"]
    columns_to_remove = [col for col in columns_to_remove if col in dialect_dataset["train"].column_names]
    dialect_dataset = dialect_dataset.remove_columns(columns_to_remove)

# Resample audio to 16kHz (Whisper's expected sampling rate)
dialect_dataset = dialect_dataset.cast_column("audio", Audio(sampling_rate=16000))

print("Dataset preprocessing completed")
print(f"First sample: {dialect_dataset['train'][0]}")

In [ ]:
def prepare_dataset(batch):
    """Prepare dataset for training by extracting features and tokenizing text."""
    # Load and resample audio data from 48 to 16kHz
    audio = batch["audio"]

    # Compute log-Mel input features from input audio array
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]

    # Encode target text to label ids
    batch["labels"] = tokenizer(batch["sentence"]).input_ids
    return batch

# Apply preprocessing to dataset
print("Applying data preparation (this may take a few minutes)...")
dialect_dataset = dialect_dataset.map(
    prepare_dataset, 
    remove_columns=dialect_dataset.column_names["train"], 
    num_proc=2
)

print(f"Preprocessed dataset: {dialect_dataset}")

## Data Collator for PEFT Training

In [ ]:
import torch
from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # Split inputs and labels since they have to be of different lengths and need different padding methods
        # First treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # Get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # Pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # Replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # If bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

# Initialize data collator
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)
print("Data collator initialized")

## Evaluation Metrics

In [ ]:
import evaluate

# Load WER metric
metric = evaluate.load("wer")

def compute_metrics(pred):
    """Compute WER metric for evaluation."""
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # Replace -100 with the pad_token_id
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # We do not want to group tokens when computing the metrics
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

print("Evaluation metrics configured")

## Load Pre-trained Model with 8-bit Quantization

In [ ]:
from transformers import WhisperForConditionalGeneration

# Load model in 8-bit for memory efficiency
print(f"Loading {model_name_or_path} with 8-bit quantization...")
model = WhisperForConditionalGeneration.from_pretrained(
    model_name_or_path, 
    load_in_8bit=True, 
    device_map="auto"
)

print(f"Model loaded: {model}")
print(f"Model parameters: {model.num_parameters():,}")

## Prepare Model for 8-bit Training

In [ ]:
from peft import prepare_model_for_int8_training

# Prepare model for 8-bit training
model = prepare_model_for_int8_training(model, output_embedding_layer_name="proj_out")

# Make inputs require grad for convolutional layers
def make_inputs_require_grad(module, input, output):
    output.requires_grad_(True)

model.model.encoder.conv1.register_forward_hook(make_inputs_require_grad)

print("Model prepared for 8-bit training")

## Apply LoRA (Low-Rank Adaptation)

In [ ]:
from peft import LoraConfig, get_peft_model

# Configure LoRA
lora_config = LoraConfig(
    r=32,  # Rank
    lora_alpha=64,  # Alpha parameter for LoRA scaling
    target_modules=["q_proj", "v_proj"],  # Target modules for LoRA
    lora_dropout=0.05,  # Dropout for LoRA layers
    bias="none",  # Bias type
)

# Apply LoRA to model
model = get_peft_model(model, lora_config)

# Print trainable parameters
model.print_trainable_parameters()

print("\nLoRA configuration applied successfully!")
print("Only training ~1% of the model parameters with LoRA adapters.")

## Training Configuration

In [ ]:
from transformers import Seq2SeqTrainingArguments

# Training arguments optimized for PEFT
training_args = Seq2SeqTrainingArguments(
    output_dir=f"./whisper-small-arabic-{target_dialect}-peft",  # Output directory
    per_device_train_batch_size=16,  # Larger batch size possible with PEFT
    gradient_accumulation_steps=1,  # Can use smaller accumulation with larger batch
    learning_rate=1e-3,  # Higher learning rate for LoRA
    warmup_steps=50,
    num_train_epochs=3,  # Fewer epochs needed with PEFT
    evaluation_strategy="steps",
    fp16=True,  # Use mixed precision
    per_device_eval_batch_size=16,
    generation_max_length=128,
    logging_steps=25,
    save_steps=500,
    eval_steps=500,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    save_total_limit=2,
    # PEFT specific settings
    remove_unused_columns=False,  # Required for PeftModel
    label_names=["labels"],  # Required for PeftModel
    push_to_hub=False,  # Set to True if you want to push to hub
)

print("Training arguments configured for PEFT")

## PEFT Training Setup

In [ ]:
from transformers import Seq2SeqTrainer, TrainerCallback, TrainingArguments, TrainerState, TrainerControl
from transformers.trainer_utils import PREFIX_CHECKPOINT_DIR
import os

# Callback to save only PEFT adapter weights
class SavePeftModelCallback(TrainerCallback):
    def on_save(
        self,
        args: TrainingArguments,
        state: TrainerState,
        control: TrainerControl,
        **kwargs,
    ):
        checkpoint_folder = os.path.join(args.output_dir, f"{PREFIX_CHECKPOINT_DIR}-{state.global_step}")

        peft_model_path = os.path.join(checkpoint_folder, "adapter_model")
        kwargs["model"].save_pretrained(peft_model_path)

        pytorch_model_path = os.path.join(checkpoint_folder, "pytorch_model.bin")
        if os.path.exists(pytorch_model_path):
            os.remove(pytorch_model_path)
        return control

# Initialize trainer
trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=dialect_dataset["train"],
    eval_dataset=dialect_dataset["test"],
    data_collator=data_collator,
    # Note: compute_metrics commented out due to INT8 training constraints
    # compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
    callbacks=[SavePeftModelCallback],
)

# Disable cache for training
model.config.use_cache = False

print("PEFT Trainer initialized and ready for training!")

## Start Training

In [ ]:
# Start training
print(f"Starting PEFT fine-tuning for {target_dialect} Arabic dialect...")
print("Training progress will be logged below.")

trainer.train()

## Save the Final Model

In [ ]:
# Save the final PEFT model
final_model_path = f"./whisper-small-arabic-{target_dialect}-peft-final"
trainer.model.save_pretrained(final_model_path)
processor.save_pretrained(final_model_path)

print(f"Final PEFT model saved to: {final_model_path}")
print(f"Model size: {os.path.getsize(final_model_path + '/adapter_model.bin') / 1024**2:.1f} MB")

# Optionally push to hub
# trainer.push_to_hub()

## Load and Test the Fine-tuned Model

In [ ]:
from peft import PeftModel, PeftConfig
from transformers import WhisperForConditionalGeneration, WhisperProcessor
import torch

# Load the fine-tuned PEFT model for inference
def load_peft_model(adapter_path, base_model_name="openai/whisper-small"):
    """Load PEFT model for inference."""
    # Load PEFT config
    peft_config = PeftConfig.from_pretrained(adapter_path)
    
    # Load base model
    base_model = WhisperForConditionalGeneration.from_pretrained(
        base_model_name, torch_dtype=torch.float16, device_map="auto"
    )
    
    # Load PEFT model
    model = PeftModel.from_pretrained(base_model, adapter_path)
    
    return model

# Load the trained model
try:
    inference_model = load_peft_model(final_model_path)
    inference_processor = WhisperProcessor.from_pretrained(final_model_path)
    
    print("PEFT model loaded successfully for inference!")
    
    # Enable cache for inference
    inference_model.config.use_cache = True
    
except Exception as e:
    print(f"Error loading model: {e}")
    print("You can load the model later using the load_peft_model function")

## Test the Model on Sample Audio

In [ ]:
# Test the model on a sample from the test set
def test_sample_audio(model, processor, test_sample):
    """Test the model on a sample audio."""
    # Prepare input
    input_features = processor(
        test_sample["audio"]["array"], 
        sampling_rate=test_sample["audio"]["sampling_rate"], 
        return_tensors="pt"
    ).input_features
    
    # Move to device
    if torch.cuda.is_available():
        input_features = input_features.cuda()
    
    # Generate prediction
    with torch.no_grad():
        predicted_ids = model.generate(input_features, max_length=128)
    
    # Decode prediction
    transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)[0]
    
    return transcription

# Test on a sample
if 'inference_model' in locals():
    test_sample = dialect_dataset["test"][0]
    
    print("Testing the fine-tuned PEFT model...")
    print(f"Original text: {test_sample.get('sentence', 'N/A')}")
    
    try:
        prediction = test_sample_audio(inference_model, inference_processor, test_sample)
        print(f"Predicted text: {prediction}")
    except Exception as e:
        print(f"Error during inference: {e}")
else:
    print("Model not loaded. Please run the previous cell successfully first.")

## Summary

This notebook demonstrated how to fine-tune Whisper for Arabic dialects using PEFT and LoRA:

### Key Benefits of PEFT Approach:
1. **Memory Efficient**: Used 8-bit quantization and LoRA to reduce memory usage
2. **Parameter Efficient**: Only trained ~1% of the model parameters
3. **Faster Training**: Higher batch sizes and faster convergence
4. **Smaller Checkpoints**: Adapter weights are ~60MB vs full model ~1.5GB
5. **Better Generalization**: Less prone to catastrophic forgetting

### Model Performance:
- The LoRA adapters are trained specifically for Arabic dialect recognition
- The approach maintains the base Whisper model's capabilities while adapting to dialect-specific patterns
- Fine-tuned model can be easily shared and deployed

### Next Steps:
1. Evaluate the model on held-out test sets
2. Compare performance with full fine-tuning
3. Experiment with different LoRA configurations (rank, alpha, target modules)
4. Train adapters for multiple dialects and combine them